## Opis zadatka

Date su **tri datoteke** koje sadrže podatke o **pristupima predavanjima** održanim u organizaciji Matematičkog fakulteta na **Webex platformi**. Podaci u datotekama dati su na mesečnom nivou (novembar, mart, april).
U odnosu na datoteke obrađene u prethodnom zadatku ove datoteke imaju dodatne atribute. **Cilj** je, kao i u prethodnom zadatku, po svakom opisanom datumu izračunati akumulativno broj prisutnih na sastanku. S obzirom na dostupnost dodatnih atributa biće razmotreno njihovo korišćenje radi dobijanja detaljnijih informacija.

### Učitavanje podataka

Podaci su izvorno dati u dva formata: **.csv** (coma separated values) i **.ods** (OpenDocument Spreadsheet Document). Međutim, izvršena je konverzija .ods datoteka u .csv datoteke i na taj način unifikovano je učitavanje. 
Za obradu podataka biće korišćen **pandas** i **numpy** moduli.

In [1]:
import pandas as pd
import numpy as np

### Pregled atributa

Sve tri datoteke imaju isti skup atributa pa se podaci spajaju u jednu datoteku i obrada podataka će biti vršena nad takvim skupom. Atributa ima **23**, a ukupan broj zapisa u sve tri datoteke je **94085**.

* **Meeting number**: broj sastanka (ostaje fiksiran za svako ponavljanje sastanka);
* **Meeting name**: naziv sastanka;
* **Conference ID**: jedinstveni identifikator sastanka (drugačije se za svako ponavljanje sastanka);
* **User name**: korisničko ime učesnika sastanka;
* **Email**: email učesnika;
* **OS**: operativni sistem koji je koristio učesnik za pristup sastanku;
* **Browser**: veb pregledač koji je koristio učesnik za pristup sastanku;
* **Location**: lokacija učesnika;
* **Join date**: datum sastanka (godina-mesec-dan);
* **Start time**: vreme pristupa učesnika sastanku (godina-mesec-dan sat-minut-sekund);
* **End time**: vreme napuštanja sastanka (godina-mesec-dan sat-minut-sekund);
* **Duration**: vreme provedeno na sastanku izraženo u minutima;
* **Client IP**: privatna IP adresa koju je učesnik koristio za pristup sastanku;
* **Gateway IP**: javna IP adresa učesnika koja je rutabilna;
* **Join meeting time**: vreme od ulaska na link sastanka do učitavanja prozora + vreme od pritiskanje Join dugmeta prozora do konektovanja na sastanak;
* **VoIP packet loss**: prosečan gubitak paketa zvučnog zapisa za vreme trajanja sastanka (u procentima);
* **VoIP latency**: prosečno kašnjenje zvuka tokom sastanka (u milisekundama);
* **TCP percentage**: udeo vremena korišćenja TCP protokola;
* **UDP percentage**: udeo vremena korišćenja UDP protokola;
* **Cmr**: ako je korisnik pristupio sastanku korišćenjem Webex Room ili Desk Device;
* **Sharing**: da li je učesnik podelio svoj ekran tokom sastanka;
* **Record**: da li je učesnik pritisnu dugme za snimanje sastanka; 
* **Video minutes**: koliko dugo je bila uključena kamera učesnika (u minutima).

In [2]:
kolone = ['Meeting number','Meeting name','Conference ID','User name', 'Email', 'Location', 'Join date', 'Start time', 
          'End time','Duration','Operating system','Browser','Client IP','Gateway IP','Join meeting time',
          'VoIP packet loss','VoIP latency','TCP percentage','UDP percentage', 'Cmr','Sharing','Record','Video minutes']

In [3]:
ucitaj = lambda datoteka: pd.read_csv(datoteka, 
                                      header=0,
                                      names=kolone,
                                      low_memory=False)

In [4]:
podaci_novembar = ucitaj('NovembarWebex Meetings_Attendees_matf.webex.com_2020-11-01_2020-11-30.csv')
podaci_mart = ucitaj('Mart2021WebexMeetingsAttendees.csv')
podaci_april = ucitaj('April2021_Webex Meetings_Attendees_matf.webex.com_2021-04-01_2021-04-30.csv')

In [5]:
podaci = pd.concat([podaci_novembar, podaci_mart, podaci_april])
podaci.reset_index(drop=True, inplace=True)

### Odbacivanje atributa

Atributi bi mogli biti **grupisani** po tome kakve informacije daju:
* O **sastanku** [Meeting number, Meeting name, Conference ID, Join date]
* O **učesniku** [User name, Email, Location]
* O **konekciji** [Operating system, Browser, Client IP, Gateway IP, VoIP packet loss, VoIP latency, TCP percentage, UDP percentage, Join meeting time]
* O **aktivnosti učesnika na sastanku** [Start time, End time, Duration, Cmr, Sharing, Record, Video minutes]

Na osnovu zadatka moguće je odmah **odbaciti** grupu atributa koja daje informacije o konekciji i o učesniku.

In [6]:
podaci.drop(columns = ['User name', 'Email', 'Location', 'Start time', 'End time','Operating system','Browser','Client IP'
                       ,'Gateway IP','Join meeting time', 'VoIP packet loss','VoIP latency','TCP percentage','UDP percentage'], 
            inplace=True)

### Tipovi atributa 

Analizom atributa i uvidom u njihove vrednosti mogu se uočiti njihovi **tipovi**, a u skladu sa tim potrebno je izvršiti određene konverzije (inicijalno je većini atributa dodeljen tip *object*).

Atributi ***Join date*, *Start time* i *End time*** bi trebalo da nose vremensku oznaku. ***Cmr*, *Sharing* i *Record*** mogu biti posmatrani kao binarni/istinitosni atributi (boolean). Međutim izvršena je drugačija izmena tih atributa kako bi se olakšala njihova upotreba u izveštaju. Vrednost atributa ***Video minutes*** je u 94.6% slučaja nedefinisana vrednost koja zapravo odgovara slučaju da učesnik nije koristio video, pa se nedefinisane vrednosti u skladu sa tim zamenjuju sa vrednošću 0 (nula). Dodat je atribut ***Video***, zasnovan na *Video minutes*, koji predstavlja indikator da li je učesnik uključivao svoju kameru.

In [7]:
podaci.dtypes

Meeting number      int64
Meeting name       object
Conference ID     float64
Join date          object
Duration           object
Cmr                object
Sharing            object
Record             object
Video minutes      object
dtype: object

In [8]:
podaci['Video minutes'].fillna(int(0), inplace=True)

Prilikom pokušaja konverzije *Join date* kolone u datum, otkriven je unos čiji format i redosled atributa nisu odgovarajući i narušavaju strukturu. Taj zapis je odbačen.

In [9]:
podaci = podaci[~(podaci['Join date'] == 'Serbia')]

In [10]:
with pd.option_context('mode.chained_assignment', None):
    
    podaci['Join date'] = podaci['Join date'].apply(pd.to_datetime)

    podaci['Meeting name'] = podaci['Meeting name'].astype('category')
    podaci['Conference ID'] = podaci['Conference ID'].astype('category')

    podaci['Duration'] = podaci['Duration'].astype('int')
    podaci['Video minutes'] = podaci['Video minutes'].astype('int')

    podaci['Cmr'] = np.where(podaci['Cmr'] == 'Y', int(1),int(0))
    podaci['Record'] = np.where(podaci['Record'] == 'Y', int(1), int(0))
    podaci['Sharing'] = np.where(podaci['Sharing'] == 'Y', int(1), int(0))
    podaci['Video'] = np.where(podaci['Video minutes'] > int(0), int(1), int(0))

## Zadatak:

Cilj se ostvaruje istim postupkom koji je korišćen u prvom zadatku. Izdvojeni podaci sadrže **broj učesnika svakog održanog sastanka uređenih po datumima**. Dodatno su izdvojeni **nazivi sastanaka**, njihovo **trajanje** i **broj učesnika koji su podelili ekran** i **broj učesnika koji su uključili video** za svaki održani sastanak. 

Ideja je bila izdvojiti i podatke o **domaćinu sastanka** na osnovu dužine (*Duration*) prisustva. Zamisao je bila za svaki sastanak izdvojiti najduže prisutnog učesnika. Naravno, takvih učesnika može biti više, pa je ideja bila da se od svih takvih učesnika izdvoji onaj koji u svom mejlu ima *@matf.bg.ac.rs*. Međutim, postoje slučajevi kada to ne funkcioniše, kao na primer kada je organizovan seminar na kom su svi učesnici profesori ili kada ne postoji učesnik koji je profesor ili kada uopšte nije u upotrebi zvanični mejl fakulteta. Zbog toga se ne može sa pouzdanošću zaključiti domaćin i to je izostavljeno.

In [11]:
naziv_sastanka = podaci[['Conference ID','Meeting name']].drop_duplicates().set_index('Conference ID')

In [12]:
trajanje_sastanka = pd.DataFrame(podaci.groupby('Conference ID', observed=True)['Duration'].max())

In [13]:
ekran_kamera = pd.DataFrame(podaci.groupby('Conference ID')[['Sharing','Video']].agg('sum'))

In [14]:
prisutnih_na_sastanku = podaci.groupby('Join date')['Conference ID'].value_counts()
prisutnih_na_sastanku = pd.DataFrame(prisutnih_na_sastanku).set_axis(['Number of attendees'], axis=1)

In [15]:
izdvojeni_podaci = prisutnih_na_sastanku.join(naziv_sastanka, on='Conference ID').join(trajanje_sastanka, on='Conference ID').join(ekran_kamera, on='Conference ID')
izdvojeni_podaci = izdvojeni_podaci[['Meeting name','Number of attendees','Sharing', 'Video']]

izdvojeni_podaci.to_csv('izdvojeni_podaci.csv')